This notebook tests which pages from SWDE are available in archive.org. We could then use those because they have also their CSS archived.

In [ ]:
import importlib
import json
import dataclasses

from tqdm.auto import tqdm

from awe.data import swde, wayback, constants
from awe import awe_graph
for module in [wayback]:
    importlib.reload(module)

In [ ]:
# Try only subset for now.
vertical = swde.VERTICALS[0]
website = vertical.websites[1]
pages = website.pages[:50]

In [ ]:
SWDE_TIMESTAMP = '20110601000000' # SWDE was released in 2011
WAYBACK_DATA_PATH = f'{constants.DATA_DIR}/wayback.json'

In [ ]:
# Load saved responses.
with open(WAYBACK_DATA_PATH, mode='r', encoding='utf-8') as f:
    data_dict = json.loads(f.read())
already_loaded = 0
newly_loaded = 0
newly_skipped = 0
not_loaded = 0
for page in pages:
    if page.archived is False:
        response = data_dict.get(page.url, False)
        if response is None:
            # `null` value saved in the JSON means the page was fetched but the
            # API returned no results.
            page.archived = None
            newly_skipped += 1
        elif response is False:
            # This means the URL is not contained in the JSON.
            not_loaded += 1
        else:
            page.archived = wayback.WaybackPage(page.url, **response)
            newly_loaded += 1
    else:
        already_loaded += 1
already_loaded, newly_loaded, newly_skipped, not_loaded, len(data_dict)

In [ ]:
# Fetch data from API.
RETRY_FAILED = False
fetched = 0
failed = 0
for page in tqdm(pages, desc='pages'):
    if page.archived is False or (page.archived is None and RETRY_FAILED):
        page.archived = wayback.WaybackPage.get(page.url, SWDE_TIMESTAMP)
        fetched += 1
        if page.archived is None:
            failed += 1
fetched, failed

In [ ]:
# Run some stats on data.
no_snapshot = 0
bad_status = 0
total = 0
for page in pages:
    if page.archived is False:
        break
    elif page.archived is None:
        no_snapshot += 1
    elif page.archived.status != 200:
        bad_status += 1
    total += 1
no_snapshot, bad_status, total

In [ ]:
# Store obtained WaybackMachine API responses in a file.
def serialize_page(page: awe_graph.HtmlPage):
    if page.archived is None:
        return None
    d = dataclasses.asdict(page.archived)
    del d['original_url']
    return d
data_dict = data_dict | {
    page.url: serialize_page(page)
    for page in pages
    if page.archived is not False
}
with open(WAYBACK_DATA_PATH, mode='w', encoding='utf-8') as f:
    f.write(json.dumps(data_dict, indent=2))
len(data_dict), WAYBACK_DATA_PATH